In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.chdir("../")

In [2]:
class Args:
    def __init__(self):
        self.toy_size = 1000
        self.toy = True
        self.max_len = 200
        self.batch_size = 8
        self.eval_batch_size = 16
        self.lr = 1e-5
        self.steps = 100000 
        self.clip = 1.0
        self.t_pos = 0.7
        self.t_mix = 0.7
        self.trade_off = 0.5
        self.lambd = 1
        self.log_step = 500
        self.log_dir = "log/scl_mix_training_t_mix_{}_t_pos_{}_lambd_{}.pkl".format(self.t_mix,self.t_pos,self.lambd)
        self.model_dir = "models/scl_mix_training.pkl"

        self.dataset_pth = "none"
        self.dataset = "amazon_2"
        self.percentage = 0.01
        self.gpu_ids = 0

args = Args()


In [3]:
import scl_data
train_dataset, test_dataset = scl_data.make_dataset(args)

100%|██████████| 2000/2000 [00:03<00:00, 514.18it/s]


In [10]:
torch.save([train_dataset,test_dataset],"dataset/amazon_polarity/processed_dataset_18K")

In [4]:
from torch.utils.data import TensorDataset, DataLoader
train_loader = DataLoader(train_dataset, num_workers=2,batch_size=args.batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, num_workers=2,batch_size=args.batch_size, shuffle=True, drop_last=False)

In [6]:
from scl_model import scl_model
from transformers import RobertaForSequenceClassification, RobertaModel, RobertaConfig
import torch
pretrain_model = RobertaForSequenceClassification.from_pretrained("roberta-base")
config = RobertaConfig.from_pretrained("roberta-base")
# device = torch.device(args.gpu_ids)
device = torch.device("cpu")
model = scl_model(config,device,pretrain_model,args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [9]:
model = model.to(device)
for batch in train_loader:
    ce_loss_x, ce_loss_s, ce_loss_semi, scl_loss, scl_loss_semi = model(batch)
    break

In [25]:
scl_loss

tensor(18.7815, device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--steps', type=int, default=10000)
parser.add_argument("--toy",action='store_true')
parser.add_argument("--local_rank",type=int, default=0)
parser.add_argument("--num_workers",type=int,default=1)
parser.add_argument("--toy_size",type=int,default=1000)
parser.add_argument("--seed",type=int,default=41)
parser.add_argument("--gpu_ids",type=int,default=0)
parser.add_argument("--batch_size", type=int, default=8)
parser.add_argument("--eval_batch_size", type=int, default=16)
parser.add_argument("--max_len",type=int, default=200)
parser.add_argument('--lr', type=float, default=1e-5)
parser.add_argument('--clip',type=float,default=1)
parser.add_argument('--t_pos',type=float,default=0.9)
parser.add_argument('--t_mix',type=float,default=0.9)
parser.add_argument('--trade_off',type=float,default=0.5)
parser.add_argument("--lambd",type=float,default=0.8)


parser.add_argument('--log_step',type=int,default=100)
parser.add_argument('--log_dir',type=str,default="finetune_log.pkl")
parser.add_argument('--model_dir',type=str,default="finetune_model.pkl")

parser.add_argument('--dataset_pth',type=str,default="none")
parser.add_argument('--dataset',type=str,default="amazon_2")
parser.add_argument('--percentage',type=float,default=0.01)

parser.add_argument('--loss_mask',type=str,default = "1,1,1,1,1")

In [12]:
!python main_scl.py --steps 100000 --gpu_ids 0 --loss_mask 1,0.5,0.2,0,0 --dataset_pth dataset/amazon_polarity/processed_dataset_18K --log_step 100 --log_dir log/run_1.pkl --model_dir run_1.pkl

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie